# Parmi les visiteurs de pages CC combien utilisent la partie « articles de la convention collective »

**Objectif** : Parmi les visiteurs des pages CC (page qui contiennent «  /convention-collective/ » dans l’url) : 
- combien cliquent sur les accordéons qui sont dans la partie « Articles de la convention collective » : 
- combien cliquent sur les « tuiles articles » qui amènent sur légifrance : 


## On commence par le chargement des données

In [1]:
import pandas as pd
import numpy as np
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict

tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

## Si on veut lire en requêtant Elastic Search

In [6]:
es_connector = ElasticsearchConnector(env='monolog')

   
QUERY = {
  "query": {
    "bool": {
      "must": [
        {
          "bool": {
            "should": [
              {
                "prefix": {
                  "url": "https://code.travail.gouv.fr/convention-collective/"
                }
              },
              {
                "term": {
                  "type": "outlink"
                }
              }
            ]
          }
        },
        {
          "range": {
            "logfile": {
              "gte": "2022-07-01",
              "lt": "2022-11-01"
            }
          }
        }
      ]
    }
  }
}

connection with ElasticSearch is KO


/Users/martial/Documents/workspace/socialgouv/cdtn-monolog/explorations/src/elasticsearch_connector.py:53: ElasticsearchWarning: The client is unable to verify that the server is Elasticsearch due security privileges on the server side
  connection_sucess = '\x1bestablished with success\x1b' if es_connection.ping() else \


In [7]:
logs = es_connector.execute_query(QUERY, "logs-new")

  0%|          | 0/1701914 [00:00<?, ?it/s]

# Début de l'exploration

## 1. Nombre de visites sur les pages convention collective

### Comptage du nombre de visiteurs uniques sur ces pages

In [31]:
unique_visits = logs[logs['url'].str.startswith('https://code.travail.gouv.fr/convention-collective/')].idVisit.nunique()
unique_visits

484651

## 2. Nombre de click sur un menu déroulant article de convention collective 

On va compter le nombre de click sur `pagecc_clicktheme`

In [9]:
nb_clicks_pagecc_clicktheme = logs[
    (logs['type']=='pagecc_clicktheme')
].idVisit.nunique()
nb_clicks_pagecc_clicktheme

110116

## 3. Events de sortie

Le problème c'est qu'on a pas les évents de sorties

In [26]:
visits_with_clicktheme = logs.groupby(by='idVisit').apply(lambda g: 'pagecc_clicktheme' in set(g.type))
visits_with_clicktheme.name = 'clicktheme'

In [65]:
visits_with_outlink = logs[logs['url'].str.startswith('https://legifrance.gouv.fr')]\
  .groupby(by='idVisit')\
  .apply(lambda g: 'outlink' in set(g.type))
visits_with_outlink.name = 'outlink'
visits_with_outlink

idVisit
19617576    True
19617599    True
19617633    True
19617638    True
19617639    True
            ... 
26159572    True
26159607    True
26159618    True
26159636    True
26159707    True
Name: outlink, Length: 122535, dtype: bool

In [66]:
visits_with_clicktheme_and_outlink = visits_with_clicktheme\
  .to_frame()\
  .merge(visits_with_outlink, left_index=True, right_index=True)\
  .apply(lambda x: x.outlink and x.clicktheme, axis = 1)\
  .value_counts().iloc[1]
visits_with_clicktheme_and_outlink

49496

### Présentation Graphique du résultat

In [69]:
data = dict(
    number=[unique_visits, nb_clicks_pagecc_clicktheme, visits_with_clicktheme_and_outlink],
    stage=["Total de visites avec au moins une CC visitée", "Au moins un click sur un accordéon", "Au moins un départ du site sur legifrance"]
)
fig = px.funnel(data, x='number', y='stage')
fig.show()